In [25]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms

from tqdm.auto import tqdm
import timm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [26]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [27]:
class PlayingCardDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data = ImageFolder(data_dir, transform=transform)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]
    
    @property
    def classes(self):
        return self.data.classes

In [28]:
data_dir='/kaggle/input/cards-image-datasetclassification/train'
dataset = PlayingCardDataset(data_dir=data_dir)

In [29]:
len(dataset)

7624

In [30]:
dataset.classes

['ace of clubs',
 'ace of diamonds',
 'ace of hearts',
 'ace of spades',
 'eight of clubs',
 'eight of diamonds',
 'eight of hearts',
 'eight of spades',
 'five of clubs',
 'five of diamonds',
 'five of hearts',
 'five of spades',
 'four of clubs',
 'four of diamonds',
 'four of hearts',
 'four of spades',
 'jack of clubs',
 'jack of diamonds',
 'jack of hearts',
 'jack of spades',
 'joker',
 'king of clubs',
 'king of diamonds',
 'king of hearts',
 'king of spades',
 'nine of clubs',
 'nine of diamonds',
 'nine of hearts',
 'nine of spades',
 'queen of clubs',
 'queen of diamonds',
 'queen of hearts',
 'queen of spades',
 'seven of clubs',
 'seven of diamonds',
 'seven of hearts',
 'seven of spades',
 'six of clubs',
 'six of diamonds',
 'six of hearts',
 'six of spades',
 'ten of clubs',
 'ten of diamonds',
 'ten of hearts',
 'ten of spades',
 'three of clubs',
 'three of diamonds',
 'three of hearts',
 'three of spades',
 'two of clubs',
 'two of diamonds',
 'two of hearts',
 'two o

In [31]:
target_to_class = {v: k for k, v in ImageFolder(data_dir).class_to_idx.items()}
target_to_class

{0: 'ace of clubs',
 1: 'ace of diamonds',
 2: 'ace of hearts',
 3: 'ace of spades',
 4: 'eight of clubs',
 5: 'eight of diamonds',
 6: 'eight of hearts',
 7: 'eight of spades',
 8: 'five of clubs',
 9: 'five of diamonds',
 10: 'five of hearts',
 11: 'five of spades',
 12: 'four of clubs',
 13: 'four of diamonds',
 14: 'four of hearts',
 15: 'four of spades',
 16: 'jack of clubs',
 17: 'jack of diamonds',
 18: 'jack of hearts',
 19: 'jack of spades',
 20: 'joker',
 21: 'king of clubs',
 22: 'king of diamonds',
 23: 'king of hearts',
 24: 'king of spades',
 25: 'nine of clubs',
 26: 'nine of diamonds',
 27: 'nine of hearts',
 28: 'nine of spades',
 29: 'queen of clubs',
 30: 'queen of diamonds',
 31: 'queen of hearts',
 32: 'queen of spades',
 33: 'seven of clubs',
 34: 'seven of diamonds',
 35: 'seven of hearts',
 36: 'seven of spades',
 37: 'six of clubs',
 38: 'six of diamonds',
 39: 'six of hearts',
 40: 'six of spades',
 41: 'ten of clubs',
 42: 'ten of diamonds',
 43: 'ten of hear

In [32]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

In [33]:
transform

Compose(
    Resize(size=(128, 128), interpolation=bilinear, max_size=None, antialias=warn)
    ToTensor()
)

In [34]:
dataset = PlayingCardDataset(data_dir=data_dir,
                             transform=transform)

In [35]:
dataset[100][0].shape

torch.Size([3, 128, 128])

In [36]:
BATCH_SIZE = 32
NUM_WORKERS = os.cpu_count()

dataloader = DataLoader(dataset, 
                        batch_size=BATCH_SIZE,
                        shuffle=True,
                        num_workers=NUM_WORKERS)

In [37]:
class SimpleCardClassifier(nn.Module):
    def __init__(self, num_classes: int):
        super().__init__()
        self.base_model = timm.create_model('efficientnet_b0', pretrained=True)
        self.features = nn.Sequential(*list(self.base_model.children())[:-1])
        
        enet_out_size = 1280
        self.classifier = nn.Linear(in_features=enet_out_size,
                                    out_features=num_classes)
        
    def forward(self, x: torch.Tensor):
        return self.classifier(self.features(x))

In [38]:
model = SimpleCardClassifier(num_classes=len(dataset.classes))
model

SimpleCardClassifier(
  (base_model): EfficientNet(
    (conv_stem): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNormAct2d(
      32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): SiLU(inplace=True)
    )
    (blocks): Sequential(
      (0): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn1): BatchNormAct2d(
            32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): SiLU(inplace=True)
          )
          (se): SqueezeExcite(
            (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (act1): SiLU(inplace=True)
            (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (gate): Sigmoid()
          )
          (conv_pw): Conv2d(32, 16, ke

In [39]:
for image_batch, label_batch in dataloader:
    break

In [40]:
output = model(image_batch)
output.shape

torch.Size([32, 53])

In [41]:
output.argmax(dim=1)

tensor([38, 49, 10, 14,  1,  6, 51, 22, 29, 48, 28, 25, 11,  9,  5, 10, 51, 22,
        25, 20, 45, 38, 12, 41, 35, 18, 32, 25, 22,  7, 22, 41])

In [42]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(),
                             lr=1e-3)

In [43]:
BATCH_SIZE = 32
NUM_WORKERS = os.cpu_count()

train_dir = '/kaggle/input/cards-image-datasetclassification/train'
test_dir = '/kaggle/input/cards-image-datasetclassification/test'
val_dir = '/kaggle/input/cards-image-datasetclassification/valid'

train_data = PlayingCardDataset(data_dir=train_dir,
                                transform=transform)
val_data = PlayingCardDataset(data_dir=val_dir,
                              transform=transform)
test_data = PlayingCardDataset(data_dir=test_dir,
                               transform=transform)

train_dataloader = DataLoader(dataset=train_data,
                              batch_size=BATCH_SIZE,
                              shuffle=True,
                              num_workers=NUM_WORKERS)
val_dataloader = DataLoader(dataset=val_data,
                            batch_size=BATCH_SIZE,
                            shuffle=False,
                            num_workers=NUM_WORKERS)
test_dataloader = DataLoader(dataset=test_data,
                             batch_size=BATCH_SIZE,
                             shuffle=False,
                             num_workers=NUM_WORKERS)

In [44]:
num_epochs = 5
train_losses, val_losses = [], []

model = SimpleCardClassifier(num_classes=len(train_data.classes)).to(device)

for epoch in range(num_epochs):
    model.train()
    train_loss, val_loss = 0, 0
    
    for X, y in tqdm(train_dataloader, desc='Training Loop'):
        X, y = X.to(device), y.to(device)
        
        preds = model(X)
        loss = criterion(preds, y)
        train_loss += loss
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    train_loss /= len(train_dataloader)
    train_losses.append(train_loss)
    
    model.eval()
    with torch.inference_mode():
        for X, y in tqdm(val_dataloader, desc='Validation Loop'):
            X, y = X.to(device), y.to(device)

            preds = model(X)
            loss = criterion(preds, y)
            val_loss += loss
        
        val_loss /= len(val_dataloader)
        val_losses.append(val_loss)
        
    print(f'Epoch: {epoch + 1} / {num_epochs} | Train Loss: {train_loss} | Validation Loss: {val_loss}')

        

Training Loop:   0%|          | 0/239 [00:00<?, ?it/s]

Validation Loop:   0%|          | 0/9 [00:00<?, ?it/s]

Epoch: 1 / 5 | Train Loss: 3.9946351051330566 | Validation Loss: 3.985377550125122


Training Loop:   0%|          | 0/239 [00:00<?, ?it/s]

Validation Loop:   0%|          | 0/9 [00:00<?, ?it/s]

...

Validation Loop:   0%|          | 0/9 [00:00<?, ?it/s]

Epoch: 4 / 5 | Train Loss: 3.994926929473877 | Validation Loss: 3.9850971698760986


Training Loop:   0%|          | 0/239 [00:00<?, ?it/s]

Validation Loop:   0%|          | 0/9 [00:00<?, ?it/s]

Epoch: 5 / 5 | Train Loss: 3.9914839267730713 | Validation Loss: 3.97872257232666
